# Stencil F90 Serial

In [5]:
%%bash
source /scratch/app/modulos/intel-psxe-2019.sh
mpiifort --version

Intel(R) Parallel Studio XE 2019 Update 3 for Linux*
Copyright (C) 2009-2019 Intel Corporation. All rights reserved.
ifort (IFORT) 19.0.3.199 20190206
Copyright (C) 1985-2019 Intel Corporation.  All rights reserved.



In [16]:
! uname -a

Linux sdumont14 3.10.0-957.el7.x86_64 #1 SMP Thu Oct 4 20:48:51 UTC 2018 x86_64 x86_64 x86_64 GNU/Linux


In [17]:
! cat /etc/redhat-release

Red Hat Enterprise Linux Server release 7.6 (Maipo)


Grava em um arquivo no disco

In [1]:
%%writefile stencil_seq.f90
! === stencil_seq.f90 - stencil versão sequencial ===
! Revisão 2020-08-11
! Programa originalmente escrito por C. R. Souza
! Adaptado por E. Furlan M.
! Parâmetros de compilação para debugging e para produção:
!   $ ifort -g -check all -fpe0 -warn -traceback -debug extended  \
!           -o stencil_seq  stencil_seq.f90
!   $ ifort -O3 -o stencil_seq  stencil_seq.f90
! Para rodar:
!   $ ./stencil_mpi
! HISTÓRICO:
! - parâmetros de linha de comando substituidos por linhas de programa, uma
!   vez que é mais fácil altera-los interativamente no Jupyter Notebook.
! - eliminadas as referências à biblioteca mpi.
! - "aold(sources(i,1)+1, sources(i,2)+1)": adicionado "+1" porque os índices
!   em fortran iniciam em 1. Sem essa correção acontecem erros de runtime por
!   violação dos limites da matriz.
! - "sources(3,:) = (/ n*4/5, n*8/9 /)": alterado o "n*8/9" para ficar simi-
!   lar à versão "C" do programa, escrita por Balaji.

program stencil
    implicit none
    integer, parameter  :: nsources=3
    integer             :: n=4800     ! nxn grid (4800)
    integer             :: energy=1   ! energy to be injected per iteration (1)
    integer             :: niters=500 ! number of iterations (500)
    integer             :: iters, i, j, size, sizeStart, sizeEnd
    integer, dimension(3, 2)        :: sources
    double precision, allocatable   :: aold(:,:), anew(:,:)
    double precision    :: t=0.0, t1=0.0, heat=0.0
            
    call cpu_time(t1)
    t = -t1
    
    size = n + 2
    sizeStart = 2
    sizeEnd = n + 1

    allocate(aold(size, size))
    allocate(anew(size, size))
    aold = 0.0
    anew = 0.0

    sources(1,:) = (/ n/2,   n/2   /)
    sources(2,:) = (/ n/3,   n/3   /)
    sources(3,:) = (/ n*4/5, n*8/9 /)   ! 8/9 conforme Balaji

    do iters = 1, niters, 2
        
        ! iteracao impar: anew <- stencil(aold)

        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                anew(i,j) = aold(i,j)/2.0 + (aold(i-1,j) + aold(i+1,j) +  &
                            aold(i,j-1) + aold(i,j+1))/4.0/2.0
            enddo
        enddo

        do i = 1, nsources
            anew(sources(i,1)+1, sources(i,2)+1) =  &
                anew(sources(i,1)+1, sources(i,2)+1) + energy
        enddo


        ! iteracao par: aold <- stencil(anew)

        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                aold(i,j) = anew(i,j)/2.0 + (anew(i-1,j) + anew(i+1,j) +  &
                            anew(i,j-1) + anew(i,j+1))/4.0/2.0
            enddo
        enddo
        
        do i = 1, nsources
            aold(sources(i,1)+1, sources(i,2)+1) =  &
                aold(sources(i,1)+1, sources(i,2)+1) + energy
        enddo

    enddo
   
    heat = 0.0
    do j = sizeStart, sizeEnd
        do i = sizeStart, sizeEnd
            heat = heat + aold(i,j)
        end do
    end do

    deallocate(aold)
    deallocate(anew)

    call cpu_time(t1)
    t = t + t1

    write(*, "('Heat = ' f0.4' | ')", advance="no") heat
    write(*, "('Tempo = 'f0.4)") t

end

Overwriting stencil_seq.f90


# Compilador GNU 4.8.5 (default do sistema operacional do SD)

In [3]:
! module list

Currently Loaded Modulefiles:
  1) anaconda3/2018.12


In [4]:
! gfortran --version

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36)
Copyright (C) 2015 Free Software Foundation, Inc.

GNU Fortran comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of GNU Fortran
under the terms of the GNU General Public License.
For more information about these matters, see the file named COPYING



In [6]:
! gfortran -v -O3  stencil_seq.f90  -o st_sq_gnu

Driving: gfortran -v -O3 stencil_seq.f90 -o st_sq_gnu -l gfortran -l m -shared-libgcc
Using built-in specs.
COLLECT_GCC=gfortran
COLLECT_LTO_WRAPPER=/usr/libexec/gcc/x86_64-redhat-linux/4.8.5/lto-wrapper
Target: x86_64-redhat-linux
Configured with: ../configure --prefix=/usr --mandir=/usr/share/man --infodir=/usr/share/info --with-bugurl=http://bugzilla.redhat.com/bugzilla --enable-bootstrap --enable-shared --enable-threads=posix --enable-checking=release --with-system-zlib --enable-__cxa_atexit --disable-libunwind-exceptions --enable-gnu-unique-object --enable-linker-build-id --with-linker-hash-style=gnu --enable-languages=c,c++,objc,obj-c++,java,fortran,ada,go,lto --enable-plugin --enable-initfini-array --disable-libgcj --with-isl=/builddir/build/BUILD/gcc-4.8.5-20150702/obj-x86_64-redhat-linux/isl-install --with-cloog=/builddir/build/BUILD/gcc-4.8.5-20150702/obj-x86_64-redhat-linux/cloog-install --enable-gnu-indirect-function --with-tune=generic --with-arch_32=x86-64 --build=x86_64-

In [7]:
! ./st_sq_gnu

Heat = 1500.0000 | Tempo = 17.6792


In [1]:
%%bash
a='st_sq_gnu'
b='/stnc/Fortran'
s='/prj/ampemi/xxxx.xxxx'$b
d='/scratch/ampemi/xxxx.xxxx'$b
cp $s/$a $d
ls -lh $d/$a

-rwxr-xr-x 1 xxxx.xxxx ampemi 13K Dec  2 22:49 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu


In [2]:
%%writefile st_sq_gnu.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo
#SBATCH --ntasks=01            #Total de tarefas
#SBATCH -J stsq                #Nome do job, 8 caracteres
#SBATCH -p cpu_dev             #Fila (partition) a ser utilizada
#SBATCH --time=00:02:00        #Tempo max. de execução 5 minutos
#SBATCH --exclusive            #Utilização exclusiva dos nós
# #SBATCH --nodes=1              #Qtd de nós
# #SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
nodeset -e $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#Entra no diretório de trabalho
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cd $d

#Configura o executavel
echo '-- executavel -------------------------'
a='st_sq_gnu'
b='/stnc/Fortran'
d='/scratch/ampemi/xxxx.xxxx'$b
EXEC1=$d/$a
echo 'Executavel: '$EXEC1

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ srun -n ' $SLURM_NTASKS $EXEC1
srun -n $SLURM_NTASKS $EXEC1
echo '-- FIM --------------------------------'

Overwriting st_sq_gnu.srm


In [3]:
! sbatch st_sq_gnu.srm

Submitted batch job 788485


In [4]:
! squeue -n stsq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788485   cpu_dev     stsq xxxx.  R       0:18      1 sdumont1090


In [5]:
! squeue -n stsq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [6]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cat $d/slurm-788485.out

- Job ID: 788485
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1090
sdumont1090
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
-- executavel -------------------------
Executavel: /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu
-- srun -------------------------------
$ srun -n  1 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu
Heat = 1500.0000 | Tempo = 19.2428
-- FIM --------------------------------


### Segunda tomada de tempo

In [7]:
! sbatch st_sq_gnu.srm

Submitted batch job 788486


In [8]:
! squeue -n stsq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788486   cpu_dev     stsq xxxx. PD       0:00      1 (None)


In [13]:
! squeue -n stsq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [14]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cat $d/slurm-788486.out

- Job ID: 788486
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1090
sdumont1090
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
-- executavel -------------------------
Executavel: /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu
-- srun -------------------------------
$ srun -n  1 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu
Heat = 1500.0000 | Tempo = 19.3187
-- FIM --------------------------------


### Terceira tomada de tempo

In [10]:
! sbatch st_sq_gnu.srm

Submitted batch job 788487


In [11]:
! squeue -n stsq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788487   cpu_dev     stsq xxxx.  R       0:04      1 sdumont1090


In [15]:
! squeue -n stsq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [17]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Fortran'
cat $d/slurm-788487.out

- Job ID: 788487
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1090
sdumont1090
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Fortran
-- executavel -------------------------
Executavel: /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu
-- srun -------------------------------
$ srun -n  1 /scratch/ampemi/xxxx.xxxx/stnc/Fortran/st_sq_gnu
Heat = 1500.0000 | Tempo = 19.1887
-- FIM --------------------------------
